## 1) Vanilla RNN - Character Level


In [18]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('../Data/input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
    
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

data has 1115394 characters, 65 unique.
----
 Bt! NXjEGwXL:GYV:YeNVUK3NxMHLzBnyeSByt!S?RC!PKdvj:EFIPuRgJ.fCitkJwUpufhpO!:qnsWNa?dn!x$jrzL.bkotVG-XMLJZzMis?:MYWdWS3JsE?Sd
bkPRAQ VdT3-?R3
XD
I'eBnXwVOQsoh uqpS' C w,'l$F jaGdgs?tUo QQiSGE.CeF:-GJjEw 
----
iter 0, loss: 104.359692
----
 i kie e  ne fOkne hnsnu knhokCE fdt snQ
 d hS khsnm AranehUezanht eny iitrqIsn$ Tua irKevlknrsiob y:e S fe haeddsl nst: k lCjrTnd ortnenz:o  newsuZdh M tnsts nnrWs?ohswe khl gn;ey grsnshk c YrB k k ,  
----
iter 100, loss: 105.477464
----
 tlankdesecwl aetyrteiuimf ho nB il
 fctg! karsr eUh mpt,,lWct keeiWohplprwe,Egkt
ehsofena$ymzleD  dbel   meyr eneo,eee Xsil,r'ena t CsBihg-ga liy,Wty:e ?nt, ttvePn ygb, a3c cIee eoovf'erusbs se,
   ac 
----
iter 200, loss: 103.678070
----
 ozb.o vvaihoHajitvr o sAnzLAe,o uCrhuBj' u.:RslseBSdaLlltnbyrh ze
aCgstub moeA,Nnue'I ou NIS?haCpyt: fix
;ealFiC,zthiwaC?nh, Av-aCSogxohgutObhM.itgdidI$owav
f;vgwen
vengl dapia,t-veasiaCYcICRv- twasta 
----
iter 300, loss: 102.022795
--

----
  an oo cimmivelane are pasd threy ume bu yorasolge
thit nutthe then thace pa, rithey urpatgeo soanth oupnrulllus
sit'.

CIIIOLANAENUSe fy that burilo theng:Clat ine g. thals bre not the
Omsh ance the  
----
iter 3400, loss: 62.229831
----
 
CIUS:
Wyo hochoqs heeses th ouigour lebres petlher betivend;
Th ruw tot ar base?
doritirs homvity;
Tore insmend ilerromor beallger he yourisererten' to ertIe, thring.
lu has vouss ongey

BR OIELUS:
P 
----
iter 3500, loss: 61.738102
----
 a
We de pivel thenetive werlidslllaic diry he ave -art
Mloceey 'atoks egro o tice pf ey muur titping dond weils of he so to moud?
Youn thaed I tidmey them aonpe nore peacewhe mbant.

MEOL:
I the beuil 
----
iter 3600, loss: 61.463238
----
 NIAUMONIUS:

COMUIlANRIOLNENIEAUC;ANNUS:
Cu'd seore aw yso fofmem' sar.
Che het, iog hy he,
cnitt; cit set, bfeet th sasd

ORUS:Whid wolcishes yuo manth thu Wif.

Sir
CaRer tilld ohil.s

COIENIAUS-MTw 
----
iter 3700, loss: 61.102512
----
  if kan mere. and yorian dowt tat 

----
 porntwiy kelr sitt tet mof ash rour kneeneanhin. I my gosshed ar all por weer. I,
I byome thaat here sith heald hoy py, hangtrour ankd, mare doltper powdeul t;
Hopes,
Wroe! irs ly weed
To to lan Tho t 
----
iter 6900, loss: 57.899356
----
 nh, me sadllo liuchuly word, norce ro dioth me plpeed vench Votv evet unU on vos, erdy kat Ttann word he dill.

FER: Gkow maks lol Iweres oy jundiner.

VOCADN:
the me keass cis he thim and, be nopmedc 
----
iter 7000, loss: 58.127223
----
 OS$EETE:
Veiklctou tis geaknh tord youe eatn but lirae,
Em
Wave matd tol ond to youur adld ot was huy hik mave lullry,
Sertas frima,,
sy has nutdmune I ulun hibs horeln nound.

CEOe Ilury ou hyelr, gh 
----
iter 7100, loss: 58.183150
----
 teres a sow, an your ar you thifl, ad wone sorlsist irn uo'r erenes, sume hou hom. 
CERS:
Hey you hand hy wid Gki and sove nise, Jinumle.
Thy.

LATBS:
De ar wimk thead site my,
Br bone bounels Go her  
----
iter 7200, loss: 58.133957
----
  winn mcathif lond by ag coy the b

----
  pivels leoth mut thy is and ow not fabtend is'r ints o wess nour TasTheer:
Wthey the dact the, pincs some of elare Ces pot hingeprer win'y ard mass thar vo dove, binsod veend bar do urais
Youd And my 
----
iter 10400, loss: 55.626017
----
 ERK:
Youd m; the rory, blistes:
Bor seand anttiw!

MANZG:
A crach mit, ro frign tfemoulgtioupy lot hese-Tot Rountene beines tin coth your urye larme ingetestin the thount ilis meaThefh pynen by and fa 
----
iter 10500, loss: 55.655356
----
  end mowith locefner'des and dadswes shesebses and that be thy yot ghere.

GRINGIZA.TI GKIN:
Angely
Aniend angare my
Invin in on o
goul wanobecow, lupy's oed histo raeerme afre that
Ceggseld Room you  
----
iter 10600, loss: 55.820562
----
  se hele ofs a depesve.

Bhe.

GKIN:
Hour wetof an ingunccemectaes.

BLY:
Mamis gloot whome coret, yoo do wir thet chave fy pill.

BR:
No we rad ank bue thoy gesture,
Pace ile hine fat'l. Be mikiir wo 
----
iter 10700, loss: 56.054313
----
 , out merds lis pranh yeloleer

----
  penish thay yourengh hou dorles bane toth Hertace shatpe I od him and romcno thakk tlayu poes foonster boame whougp to and broer,
Wher dot chend's shave woay, an heple,
nis
Of on oull,
And sory't
Tho 
----
iter 13800, loss: 56.011912
----
 I no noth Breeerf thy of ford tut.
Shy he arad,
Borwy priansthash in thus thit bee gort'd wisw hichrene,
Kort tout ave, wons fare angener mis thow thath mus mareorsole bet of oy os the glas beimy urpe 
----
iter 13900, loss: 55.910855
----
 !

OL:
Wo fostreg aned alld tous thes of bamt, mher hameh
I here wan shesint
lyc'.
Thou belt alout ond man fisteinn; mar youll thir srones mys bore mnit my agh heasn gast wall gear serdmedyy moneod ba 
----
iter 14000, loss: 55.889213
----
  novely outttere aged cend'n thoureraly here
By spatccyowist my polwsttily in blist co blive shaist mow,
Alganstt shos now out nock?
Wheres in thilg the beingtint, benmetn port wibnd! now'd by bechole 
----
iter 14100, loss: 55.610035
----
 fheuryal wat Mors;
prood
Se do

...

----
 GBAPETINAN3O:
Whize.

ANSONA:
To thed yous my prold, mu ight thas boand
gen sear.

DANIO:
Bagftend and doabl.

ION:
Whald ruir!
Wen my steminh mee than mister frepriof musonko thos im hetslo love food 
----
iter 44400, loss: 51.639962
----
 f theth
I, yif tat a hanbous nofed aming;
Plaiss leaveds
diency an:
Apld gnot, thaw, Kacl: a'tach what wellt be pelt in wolk, thir wath, ceo:
mow' Baughceou!

PEMIO:
Meate yous ech lies,
You tir.

NOL 
----
iter 44500, loss: 51.625062
----
 L:
Yove thy ly cott oe benenaon,
Heare and teling muld.

QENIO:
Norme, to nosper, bronger theed in this yor bed, deardt you fo I the donelas if ood wonles you ir lond;
Ond-me sonutione, fentele rake I 
----
iter 44600, loss: 51.591232
----
  plive your fnouk vingmow cond Is dop more.
 in maldck! so love the bome.

VI:

DUCEWAMYO:
Bnoue ald whifiWime;:
Cer ehy to then declots ood ard tor ow und.

GRETASTI ACA:
Wid is frume fours, witionge 
----
iter 44700, loss: 51.919802
----
 d gorthy bubmpell thit a mert 

----
 st.

MINIUS:
but prat,
And-sh with coe:
He anconl hibl:
O'couns other theiun u porn.

FENIUS:
Ands in'lf 'Stir,
Hoth and this yow you tith in the saed's warce'd mone: weiSizer: ande nare in lipterct,  
----
iter 47800, loss: 50.717208
----
  whall ho recg Goodcurt of to sepcave urlird beaf
Fare of if dadisu,
Phuld awh helens shang. 'lise beas hing ander: reass r$aly is he livegeof Rofrce souns thip;
Abers Cold thomenld hes is, lance, it  
----
iter 47900, loss: 50.606612
----
 best, the me aw, wouctsre slay!
And Clowe. Co verst thensrefr
On
Tnistiod hand po entivet.

Shacioun noul ary.

Srivo: he.

MINIUS: Mquat gode!
On dirad?

getheet ean parlme, prisild coore!

MENENIUS: 
----
iter 48000, loss: 50.401638
----
  you hat my hene thead!

TheO fllay wear hever sobmanous ther, wgri your go.

ILONIN:
The speroult: bivestents; I
Treienter, you pordternce nobedsit the sasm;
Bust,
Inson
Kinf not-'s badoul that soust 
----
iter 48100, loss: 50.162996
----
 
No miding
, to ta an olm sile

----
 ffinss us the fold no eatlen heis, eis not; butbeartist Vorkngel. But sombent of wile what?

SLARUTIUS:
Bord!

JIWID:
I whan rould mariut 'llith nothtime ta treast thim co locked. co'shend thung I tat 
----
iter 51200, loss: 51.470181
----
  'te'st eixne r't we'lles, pherk!

VORIO:
Petrang then yell forth weack, sepunto,
Lar fringhy thwes
shee:
Whe townt then buth; thob do of pried mlothtse with hafnt as che eien. Sed ande, foltu!

Tol:
 
----
iter 51300, loss: 51.296009
----
 ot dim ly ole
Hest flainer they, not stlichansliTn the dace woodrrowl
It not theve,
Wlith wile feided jeangpa,
Buch ?fou the lordion:
Dogd eard andbly: preughte hecd stye the sfo ellet,
And wher telat 
----
iter 51400, loss: 51.004481
----
 d Epond.

MENENTEN:
Ls my lige butel ane-sert tould a haw that keds anTisp not eartwlrde this go houthare tas for to cendes.

LAALLHUHE:
Buthe wese thiw
forbect hie fay!

MOATEO:
Is gre; a fifen o'st. 
----
iter 51500, loss: 50.687231
----
 f in.

COrd nop ond mint,
May 

----
 r'd yit
No tith bllee.

GLOUCSAS:
My harionthae hath apwerrd swive dead fepster'd in move his stan heve fachit stee wenguitning ceder I didelore I God caad the blond councowe-widinghall, othin this yo 
----
iter 54600, loss: 49.691319
----
 , fool o';
Pearcut troufttierts Rodkes,
Thy sove tis depundly you bust ngoft lupe, dover:
Thaf weld: my a tlucd that dice, mage an,
We he, frinn in'r hiw with iend: fraings the cace a gfos ig lible?
P 
----
iter 54700, loss: 49.713312
----
 oustert-
Theef.--
Wole well yous unour of Our.
'spran.

TGNGES:
Youst thee I the deine wels.

COMENGELt:
The loos.
Is ace wased bese higeer is Larke,
Bogh gee sro, to hand surt, raRgh!

By
Wake theoro 
----
iter 54800, loss: 49.683743
----
 the
cres, my olesaly hard-lood that on s!

LOUCHYIO:
le rring, thin but he Mupbref it your tobs: espeet at no dor, livnt, to me, gow;
May cond fpall'th Clootho uchas, wis cound and insursm shase
The r 
----
iter 54900, loss: 49.754712
----
  to Gralsoul,
I to meating't
W

KeyboardInterrupt: 

## 2) Word Level

In [3]:
import string

# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# turn a doc into clean tokens
def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

in_filename = '../Data/republic.txt'
doc = load_doc(in_filename)
print(doc[:200])
 
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
 
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = '../Data/republic_sequences.txt'
save_doc(sequences, out_filename)

﻿I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner 
['went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced

In [2]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    file.close()
    return text


# load
in_filename = '../Data/republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Using TensorFlow backend.


TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'